# Ubiome data analysis - draft

## Introduction

this notebook is being written as part of the Mozilla Global Sprint 2018.  
It aims at providing to openhumans members with a walkthrough on how to analyse their microbiome data from ubiome.



## Data import

First, we load and install the necessary packages (be patient!):

In [138]:
source('http://bioconductor.org/biocLite.R')
biocLite('phyloseq')

Bioconductor version 3.6 (BiocInstaller 1.28.0), ?biocLite for help
A new version of Bioconductor is available after installing the most recent
  version of R; see http://bioconductor.org/install
BioC_mirror: https://bioconductor.org
Using Bioconductor 3.6 (BiocInstaller 1.28.0), R 3.4.1 (2017-06-30).
Installing package(s) ‘phyloseq’
Updating HTML index of packages in '.Library'
Making 'packages.html' ... done
Old packages: 'bindrcpp', 'bit64', 'config', 'DBI', 'dbplyr', 'ddalpha',
  'digest', 'forecast', 'foreign', 'git2r', 'hms', 'htmlwidgets', 'httpuv',
  'kernlab', 'lattice', 'lmtest', 'lubridate', 'MASS', 'Matrix', 'nlme',
  'pbdZMQ', 'pillar', 'plogr', 'prodlim', 'psych', 'randomForest', 'Rcpp',
  'RcppArmadillo', 'RCurl', 'readxl', 'repr', 'rmarkdown', 'robustbase',
  'RSQLite', 'selectr', 'sfsmisc', 'sourcetools', 'sparklyr', 'stringi',
  'survival', 'tidyverse', 'timeDate', 'tseries', 'viridisLite', 'withr',
  'xml2', 'yaml'


In [113]:
library(httr)
library(tidyverse)
library(phyloseq)

Updating HTML index of packages in '.Library'
Making 'packages.html' ... done
Bioconductor version 3.6 (BiocInstaller 1.28.0), ?biocLite for help
A new version of Bioconductor is available after installing the most recent
  version of R; see http://bioconductor.org/install
BioC_mirror: https://bioconductor.org
Using Bioconductor 3.6 (BiocInstaller 1.28.0), R 3.4.1 (2017-06-30).
Installing package(s) ‘phyloseq’
also installing the dependencies ‘zlibbioc’, ‘rhdf5’, ‘S4Vectors’, ‘IRanges’, ‘XVector’, ‘permute’, ‘mgcv’, ‘ade4’, ‘ape’, ‘Biobase’, ‘BiocGenerics’, ‘biomformat’, ‘Biostrings’, ‘cluster’, ‘data.table’, ‘igraph’, ‘multtest’, ‘vegan’

Updating HTML index of packages in '.Library'
Making 'packages.html' ... done
Old packages: 'bindrcpp', 'bit64', 'config', 'DBI', 'dbplyr', 'ddalpha',
  'digest', 'forecast', 'foreign', 'git2r', 'hms', 'htmlwidgets', 'httpuv',
  'kernlab', 'lattice', 'lmtest', 'lubridate', 'MASS', 'Matrix', 'nlme',
  'pbdZMQ', 'pillar', 'plogr', 'prodlim', 'psych', '

I do not have my own Ubiome data, so I'll import a public dataset using the public API

In [110]:
url <- paste0("https://www.openhumans.org/api/",
             "public-data/?source=direct-sharing-132")
resp <- GET(url)
results <- content(resp, as = "parsed", type = "application/json")$results
results <- bind_rows(lapply(results, as.data.frame.list, stringsAsFactors=FALSE))

The Ubiome data is apparently available as a raw fastq file or a json result file  
We'll first explore the content of the json files.  
We are not interested in the names of the users, so let's "anonimize" our dataframe first by dropping some columns

In [111]:
jsons  <- filter(results, metadata.tags..json. == 'json') %>%
    select(-metadata.tags..fastq., -user.name, -user.username,
          -metadata.tags..uBiome., -metadata.tags..16S.,
          -source, -metadata.description, -metadata.tags..json.)

In [112]:
head(jsons)

id,basename,created,download_url,user.id,metadata.user_notes
37844,taxonomy.json,2018-04-10T19:21:12.089631Z,https://www.openhumans.org/data-management/datafile-download/37844/,08868768,NA
37870,taxonomy-1.json,2018-04-10T19:21:42.698608Z,https://www.openhumans.org/data-management/datafile-download/37870/,35269294,GENITAL (F) Sampled on 12/03/2015 Kit No. 407-018-198 Registered on 12/03/2015
37872,taxonomy-2.json,2018-04-10T19:21:42.709555Z,https://www.openhumans.org/data-management/datafile-download/37872/,35269294,GENITAL (F) Sampled on 5/09/2016 Kit No. 620-054-275 Registered on 5/10/2016
37874,taxonomy-3.json,2018-04-10T19:21:42.723138Z,https://www.openhumans.org/data-management/datafile-download/37874/,35269294,SKIN Sampled on 5/09/2016 Kit No. 620-054-275 Registered on 5/10/2016 EXPLORE SAMPLE
37876,taxonomy-4.json,2018-04-10T19:21:42.734174Z,https://www.openhumans.org/data-management/datafile-download/37876/,35269294,MOUTH SET DATE Kit No. 938-066-987 Registered on 7/31/2016
37878,taxonomy-5.json,2018-04-10T19:21:42.746854Z,https://www.openhumans.org/data-management/datafile-download/37878/,35269294,GUT SET DATE Kit No. 938-066-987 Registered on 7/31/2016


We download the first json file and try to import it

In [140]:
download.file(jsons$download_url[1], destfile = "test.json")
jsonlite::fromJSON("test.json")

ERROR: Error in parse_con(txt, bigint_as_char): lexical error: invalid char in json text.
                                    {  download_time_utc: "2016-04-21T
                     (right here) ------^

